# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.4.1'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [5]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [9]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))

In [11]:
#ann.add(tf.keras.layers.Dense(units=12, activation='relu'))

### Adding the second hidden layer

In [12]:
#ann.add(tf.keras.layers.Dense(units=12, activation='relu'))

In [13]:
ann.add(tf.keras.layers.Dense(32, kernel_initializer='lecun_normal', activation='selu'))

### Adding the output layer

In [14]:
#ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [15]:
ann.add(tf.keras.layers.Dense(16, kernel_initializer='lecun_normal', activation='selu'))
ann.add(tf.keras.layers.Dense(10, activation='softmax'))

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 465us/step - loss: 0.5851 - accuracy: 0.7651
Epoch 2/100
250/250 [==============================] - 0s 449us/step - loss: 0.4756 - accuracy: 0.8277
Epoch 3/100
250/250 [==============================] - 0s 457us/step - loss: 0.4416 - accuracy: 0.8330
Epoch 4/100
250/250 [==============================] - 0s 459us/step - loss: 0.4184 - accuracy: 0.8403
Epoch 5/100
250/250 [==============================] - 0s 450us/step - loss: 0.3950 - accuracy: 0.8485
Epoch 6/100
250/250 [==============================] - 0s 446us/step - loss: 0.3812 - accuracy: 0.8559
Epoch 7/100
250/250 [==============================] - 0s 451us/step - loss: 0.3766 - accuracy: 0.8568
Epoch 8/100
250/250 [==============================] - 0s 453us/step - loss: 0.3553 - accuracy: 0.8632
Epoch 9/100
250/250 [==============================] - 0s 454us/step - loss: 0.3592 - accuracy: 0.8615
Epoch 10/100
250/250 [==============================] - 0s 446us/step - l

250/250 [==============================] - 0s 453us/step - loss: 0.2619 - accuracy: 0.8945
Epoch 80/100
250/250 [==============================] - 0s 446us/step - loss: 0.2696 - accuracy: 0.8894
Epoch 81/100
250/250 [==============================] - 0s 452us/step - loss: 0.2658 - accuracy: 0.8940
Epoch 82/100
250/250 [==============================] - 0s 453us/step - loss: 0.2593 - accuracy: 0.8935
Epoch 83/100
250/250 [==============================] - 0s 447us/step - loss: 0.2605 - accuracy: 0.8937
Epoch 84/100
250/250 [==============================] - 0s 442us/step - loss: 0.2600 - accuracy: 0.8921
Epoch 85/100
250/250 [==============================] - 0s 450us/step - loss: 0.2568 - accuracy: 0.8942
Epoch 86/100
250/250 [==============================] - 0s 452us/step - loss: 0.2576 - accuracy: 0.8961
Epoch 87/100
250/250 [==============================] - 0s 456us/step - loss: 0.2686 - accuracy: 0.8894
Epoch 88/100
250/250 [==============================] - 0s 455us/step - loss:

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [19]:
y_pred = ann.predict(X_test)
y_pred

array([[0.11474258],
       [0.14021158],
       [0.0147979 ],
       ...,
       [0.01556063],
       [0.20454517],
       [0.05762696]], dtype=float32)

In [20]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1472  123]
 [ 190  215]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)

In [22]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from keras.layers import Dense

In [23]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifer.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifer

In [24]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)

In [25]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)

In [26]:
mean = accuracies.mean()

In [27]:
variance = accuracies.std()

In [28]:
from keras.layers import Dropout

In [29]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [30]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print(mean, variance)

0.8119999945163727 0.011742023611561611


In [31]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense

In [32]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu', input_dim = 12))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'selu'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [33]:
classifier = KerasClassifier(build_fn = build_classifier)

In [34]:
parameters = {'batch_size': [10, 25, 32],
              'epochs': [50, 100],
              'optimizer': ['adam', 'rmsprop']}

In [35]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [36]:
grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

In [37]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/50
720/720 [==============================] - 0s 341us/step - loss: 0.5520 - accuracy: 0.8041
Epoch 2/50
720/720 [==============================] - 0s 347us/step - loss: 0.4399 - accuracy: 0.8108
Epoch 3/50
720/720 [==============================] - 0s 344us/step - loss: 0.4240 - accuracy: 0.8193
Epoch 4/50
720/720 [==============================] - 0s 343us/step - loss: 0.4409 - accuracy: 0.8028
Epoch 5/50
720/720 [==============================] - 0s 345us/step - loss: 0.4415 - accuracy: 0.8051
Epoch 6/50
720/720 [==============================] - 0s 344us/step - loss: 0.4277 - accuracy: 0.8129
Epoch 7/50
720/720 [==============================] - 0s 357us/step - loss: 0.4418 - accuracy: 0.8086
Epoch 8/50
720/720 [==============================] - 0s 354us/step - loss: 0.4267 - accuracy: 0.8193
Epoch 9/50
720/720 [==============================] - 0s 371us/step - loss: 0.4461 - accuracy: 0.8023
Epoch 10/50
720/720 [==============================] - 0s 344us/step - loss: 0.432

/home/andrelucax/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


720/720 [==============================] - 0s 358us/step - loss: 0.5519 - accuracy: 0.7926
Epoch 2/50
720/720 [==============================] - 0s 348us/step - loss: 0.4273 - accuracy: 0.8171
Epoch 3/50
720/720 [==============================] - 0s 346us/step - loss: 0.4185 - accuracy: 0.8237
Epoch 4/50
720/720 [==============================] - 0s 351us/step - loss: 0.4325 - accuracy: 0.8149
Epoch 5/50
720/720 [==============================] - 0s 356us/step - loss: 0.4317 - accuracy: 0.8142
Epoch 6/50
720/720 [==============================] - 0s 352us/step - loss: 0.4359 - accuracy: 0.8121
Epoch 7/50
720/720 [==============================] - 0s 357us/step - loss: 0.4307 - accuracy: 0.8150
Epoch 8/50
720/720 [==============================] - 0s 344us/step - loss: 0.4269 - accuracy: 0.8092
Epoch 9/50
720/720 [==============================] - 0s 357us/step - loss: 0.4293 - accuracy: 0.8135
Epoch 10/50
720/720 [==============================] - 0s 349us/step - loss: 0.4375 - accurac

/home/andrelucax/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


720/720 [==============================] - 0s 362us/step - loss: 0.5559 - accuracy: 0.7969
Epoch 2/50
720/720 [==============================] - 0s 364us/step - loss: 0.4370 - accuracy: 0.8099
Epoch 3/50
720/720 [==============================] - 0s 355us/step - loss: 0.4347 - accuracy: 0.8097
Epoch 4/50
720/720 [==============================] - 0s 346us/step - loss: 0.4289 - accuracy: 0.8142
Epoch 5/50
720/720 [==============================] - 0s 345us/step - loss: 0.4226 - accuracy: 0.8204
Epoch 6/50
720/720 [==============================] - 0s 345us/step - loss: 0.4392 - accuracy: 0.8064
Epoch 7/50
720/720 [==============================] - 0s 351us/step - loss: 0.4353 - accuracy: 0.8138
Epoch 8/50
720/720 [==============================] - 0s 353us/step - loss: 0.4286 - accuracy: 0.8143
Epoch 9/50
720/720 [==============================] - 0s 354us/step - loss: 0.4213 - accuracy: 0.8154
Epoch 10/50
720/720 [==============================] - 0s 351us/step - loss: 0.4278 - accurac

...

225/225 [==============================] - 0s 537us/step - loss: 0.6307 - accuracy: 0.8006
Epoch 2/100
225/225 [==============================] - 0s 426us/step - loss: 0.4458 - accuracy: 0.8056
Epoch 3/100
225/225 [==============================] - 0s 442us/step - loss: 0.4504 - accuracy: 0.8031
Epoch 4/100
225/225 [==============================] - 0s 493us/step - loss: 0.4453 - accuracy: 0.8019
Epoch 5/100
225/225 [==============================] - 0s 465us/step - loss: 0.4421 - accuracy: 0.8042
Epoch 6/100
225/225 [==============================] - 0s 468us/step - loss: 0.4474 - accuracy: 0.8012
Epoch 7/100
225/225 [==============================] - 0s 433us/step - loss: 0.4425 - accuracy: 0.8056
Epoch 8/100
225/225 [==============================] - 0s 382us/step - loss: 0.4362 - accuracy: 0.8157
Epoch 9/100
225/225 [==============================] - 0s 375us/step - loss: 0.4330 - accuracy: 0.8087
Epoch 10/100
225/225 [==============================] - 0s 383us/step - loss: 0.4407 

Epoch 80/100
225/225 [==============================] - 0s 478us/step - loss: 0.4203 - accuracy: 0.8241
Epoch 81/100
225/225 [==============================] - 0s 451us/step - loss: 0.4307 - accuracy: 0.8150
Epoch 82/100
225/225 [==============================] - 0s 474us/step - loss: 0.4262 - accuracy: 0.8133
Epoch 83/100
225/225 [==============================] - 0s 468us/step - loss: 0.4291 - accuracy: 0.8195
Epoch 84/100
225/225 [==============================] - 0s 422us/step - loss: 0.4187 - accuracy: 0.8223
Epoch 85/100
225/225 [==============================] - 0s 399us/step - loss: 0.4295 - accuracy: 0.8198
Epoch 86/100
225/225 [==============================] - 0s 407us/step - loss: 0.4215 - accuracy: 0.8195
Epoch 87/100
225/225 [==============================] - 0s 437us/step - loss: 0.4271 - accuracy: 0.8194
Epoch 88/100
225/225 [==============================] - 0s 402us/step - loss: 0.4379 - accuracy: 0.8137
Epoch 89/100
225/225 [==============================] - 0s 410us

/home/andrelucax/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


800/800 [==============================] - 1s 458us/step - loss: 0.5436 - accuracy: 0.8035
Epoch 2/100
800/800 [==============================] - 0s 392us/step - loss: 0.4374 - accuracy: 0.8085
Epoch 3/100
800/800 [==============================] - 0s 414us/step - loss: 0.4380 - accuracy: 0.8112
Epoch 4/100
800/800 [==============================] - 0s 459us/step - loss: 0.4397 - accuracy: 0.8110
Epoch 5/100
800/800 [==============================] - 0s 455us/step - loss: 0.4450 - accuracy: 0.8042
Epoch 6/100
800/800 [==============================] - 0s 408us/step - loss: 0.4271 - accuracy: 0.8156
Epoch 7/100
800/800 [==============================] - 0s 374us/step - loss: 0.4253 - accuracy: 0.8200
Epoch 8/100
800/800 [==============================] - 0s 369us/step - loss: 0.4273 - accuracy: 0.8194
Epoch 9/100
800/800 [==============================] - 0s 391us/step - loss: 0.4304 - accuracy: 0.8157
Epoch 10/100
800/800 [==============================] - 0s 435us/step - loss: 0.4348 

Epoch 80/100
800/800 [==============================] - 0s 396us/step - loss: 0.3854 - accuracy: 0.8345
Epoch 81/100
800/800 [==============================] - 0s 369us/step - loss: 0.3755 - accuracy: 0.8399
Epoch 82/100
800/800 [==============================] - 0s 353us/step - loss: 0.3797 - accuracy: 0.8389
Epoch 83/100
800/800 [==============================] - 0s 345us/step - loss: 0.3802 - accuracy: 0.8325
Epoch 84/100
800/800 [==============================] - 0s 381us/step - loss: 0.3751 - accuracy: 0.8465
Epoch 85/100
800/800 [==============================] - 0s 401us/step - loss: 0.3834 - accuracy: 0.8353
Epoch 86/100
800/800 [==============================] - 0s 382us/step - loss: 0.3709 - accuracy: 0.8440
Epoch 87/100
800/800 [==============================] - 0s 396us/step - loss: 0.3790 - accuracy: 0.8424
Epoch 88/100
800/800 [==============================] - 0s 351us/step - loss: 0.3754 - accuracy: 0.8417
Epoch 89/100
800/800 [==============================] - 0s 401us

In [39]:
best_parameters = grid_search.best_params_
best_acuracy = grid_search.best_score_ 

In [40]:
print('Best parameters: \n',best_parameters,'\nBest acuracy: \n', best_acuracy)

Best parameters: 
 {'batch_size': 10, 'epochs': 100, 'optimizer': 'adam'} 
Best acuracy: 
 0.8568749999999999
